In [3]:
import os
import pandas as pd
from src.utils import read_config
import google.generativeai as genai
import google.ai.generativelanguage as glm

from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.faiss import FAISS, DistanceStrategy
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [4]:
organized_result = pd.read_csv('temp_result/organized_result_1.csv')
organized_result.shape

(76, 5)

In [5]:
configs = read_config('.env/configs.json')
api_key =configs['g_key']
os.environ["GOOGLE_API_KEY"] = api_key
genai.configure(api_key=api_key)

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 500,
    length_function= len
)

In [7]:
reason_loader = DataFrameLoader(organized_result, page_content_column='reason')
reason_data = reason_loader.load()

target_loader = DataFrameLoader(organized_result, page_content_column='target')
target_data = target_loader.load()

reason_documents = text_splitter.transform_documents(reason_data)
print("已為所有文件進行 chunk ",len(reason_documents),"筆")

已為所有文件進行 chunk  76 筆


In [8]:
model_name = "sentence-transformers/distiluse-base-multilingual-cased-v1"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

c:\Users\sean.chang\AppData\Local\anaconda3\envs\project_envelope\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [82]:
vector_db_path = "./vector_db/"
chroma_collection_name = "langchain"
if not os.path.exists(vector_db_path):
    os.makedirs(vector_db_path)
    print(f"Created folder: {vector_db_path}")
    
vector_store = FAISS.from_documents(reason_documents, embeddings, normalize_L2=True, distance_strategy=DistanceStrategy.MAX_INNER_PRODUCT)
vector_store.save_local(folder_path=vector_db_path)

# load FAISS
vectorstore = FAISS.load_local(folder_path=vector_db_path, allow_dangerous_deserialization=True, embeddings=embeddings)

c:\Users\sean.chang\AppData\Local\anaconda3\envs\project_envelope\lib\site-packages\langchain_community\vectorstores\faiss.py:121: UserWarning: Normalizing L2 is not applicable for metric type: MAX_INNER_PRODUCT
  warnings.warn(


In [12]:
vector_store = FAISS.from_documents(reason_documents, embeddings, normalize_L2=True, distance_strategy=DistanceStrategy.MAX_INNER_PRODUCT)
vector_store.save_local(folder_path=vector_db_path)

# load FAISS
vectorstore = FAISS.load_local(folder_path=vector_db_path, allow_dangerous_deserialization=True, embeddings=embeddings)

In [54]:
query = "行政院原住民委員會的相關資訊"

embedding_vector = embeddings.embed_query(query)
docs = vectorstore.similarity_search_with_score_by_vector(embedding_vector, k=2)
docs

[(Document(page_content='行政院客家委員會、行政院原住民族委員會及僑務委員會將客家電視、原住民電視及臺灣宏觀電視等頻道節目之製播，依政府採購法委由財團法人公共電視文化事業基金會辦理，雖非無據，惟造成公視基金會與客委會、原民會及僑委會針對上開頻道節目之製播各有立場、互生歧見，致紛爭不斷，詎行政院新聞局坐視上開爭端，非但未積極研修法令，亦未尋求妥適解決辦法，核有怠失，爰依法提案糾正', metadata={'document': '52587.pdf', 'target': '行政院新聞局', 'fact': '行政院客家委員會、行政院原住民族委員會及僑務委員會(下稱客委會、原民會及僑委會)將客家電視、原住民電視及臺灣宏觀電視(下稱客視、原視及宏觀電視)等頻道節目之製播，依政府採購法委由財團法人公共電視文化事業基金會(下稱公視基金會)辦理，雖非無據，惟造成公視基金會與客委會、原民會及僑委會針對上開頻道節目之製播各有立場、互生歧見，致紛爭不斷，詎行政院新聞局(下稱新聞局)坐視上開爭端，非但未積極研修法令，亦未尋求妥適解決辦法，核有怠失，茲提出事實與理由於后一、據陳訴人指陳略以，95年無線公股處理條例於立法院三讀通過，目的在使黨、政、軍退出媒體，95年7月17日行政院召開「研商『無電視事業公股處理條例』規範原民台、客家台及宏觀頻道交由公共電視台製播2相關機關間溝通協調機制之建立問題」會議，會中決議「其製播所需經費之移撥(無政府採購法之適用問題)…」詎客委會、原民會及僑委會未依該決議辦理，仍依政府採購法，以標案形式由公視基金會辦理，實非合理又公視基金會辦理「原、客、宏觀頻道之計畫與製播」，並無獨立之經費得以辦理，而係由政府部門辦理審查、驗收，始得請領款項之情況，違反無線公股處理條例黨政軍退出媒體之精神，造成公共電視新聞自主性、專業性之箝制，詎該公股處理條例對「經費」疏漏等問題，迄未立法補正，亦造成公共電視台(下稱公視)財務經營之困境目前公視代為製播「原、客、宏觀頻道」，亦遭客委會、原民會及僑委會等延宕付款，更造成公視財務經營困難等語云云二、依據「無線電視事業公股處理條例」第14條第3項所定「政府編列預算招標採購或設置之客家電視、原住民電視、台灣宏觀電視等頻道節目之製播，應於本條例公布施行後之次年度起，『交由』公視基金會辦理」目前公視基金會係依照政府採購

In [60]:
docs[1][0].metadata

{'document': '37008.pdf',
 'target': '行政院國軍退除役官兵輔導委員會',
 'fact': '行政院國軍退除役官兵輔導委員會﹙以下簡稱退輔會﹚為改善所屬十一所榮民醫院及十八所安養機構之廢水、廢棄物處理及公共安全，以期符合行政院環保署及衛生署訂頒有關法規標準，擬訂「環境保護及公共安全計畫」，其中新建所屬單位污水處理廠及衛生下水道工程乙項，為配合八十八年度行政院推動「擴大內需方案」，併入該會「榮民安養、養護設施﹙備﹚改善中程計畫」，預算為七億六、四五六萬八千元，報經行政院八十七年十月七日台八七防字第四九四四一號函同意辦理迄八十八年下半年及八十九年度止，計有桃園榮譽國民之家等七單位之新建污水處理廠及衛生下水道工程完工經審計部派員調查其辦理情形，發現退輔會以確保排放水質符合排放標準等為由，要求所屬單位統一採用SB二R-CASS污水處理專利設備，並同意其中台北榮家等三單位不當擴增發包範圍，均有違失，謹臚陳如次一、退輔會未經蒐集商情資訊，即片面指定採購專利設備，決策過程草率依據退輔會八十一年五月廿五日函頒「行政院國軍退除役官兵輔導委員會採購作業規定」第十九條規定「採購財物預算達審計部稽察一定金額十分之一﹙五百萬元﹚以上者，以公告招標為原則」同作業規定第三十七條規定︰「採購財物，有下列情形之一者，得以議價方式辦理之︰…二、購置財物，在同一地區內，經調查僅有一家廠商出售，或無完全相同規範可資比較者三、購置定製財物，係屬專利品，獨家製造、國內詴驗製造，或原廠牌之配件，不能以他項財物替代者…」本案退輔會各安養單位為改善園區排放污水水質，俾符合環保標準，在國內已有諸多廠商具污水處理設備產製能力之情況下，理應委由技術顧問機構廣泛蒐集相關商情資訊，以為採購參考，並考量環保法規、設備功能、效益及需求等層面，妥適訂定招標規範，以確保採購品質如採購設備不符前揭採購作業規定第三十七條規定之議價條件與理由，則應依據同作業規定第十九條之規定，以公告招標方式辦理查退輔會所屬各安養單位自八十八年度起辦理污水處理廠工程，有關污水處理設備，均依據該會八十七年十月三日函發「污水處理廠規劃設計準則」之規定，指定SBR-CASS污水處理設備專利廠牌惟退輔會函發上開設計準則前，並未完整蒐集並分析比較市場其他各種污水處理設備商情資訊，以決定其採購招標方式，僅邀集所屬各單位承辦人員，於

In [ ]:
# query = "查詢: 行政院原住民委員會的採購案"
# embedding_vector = embeddings.embed_query(query)
# docs = vectorstore.similarity_search_by_vector(embedding_vector, k=1)

# match_content_list = []
# for page in docs:
#     match_content_list.append(page.page_content)
#     print(page.page_content)
#     print(page.metadata["target"])
#     print(page.metadata['document'])

行政院客家委員會、行政院原住民族委員會及僑務委員會將客家電視、原住民電視及臺灣宏觀電視等頻道節目之製播，依政府採購法委由財團法人公共電視文化事業基金會辦理，雖非無據，惟造成公視基金會與客委會、原民會及僑委會針對上開頻道節目之製播各有立場、互生歧見，致紛爭不斷，詎行政院新聞局坐視上開爭端，非但未積極研修法令，亦未尋求妥適解決辦法，核有怠失，爰依法提案糾正
行政院新聞局
52587.pdf


In [24]:
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)

llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    convert_system_message_to_human=True,
    safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },
)

In [25]:
def query_similarity_content(query, top_k=1):
    embedding_vector = embeddings.embed_query(query)
    docs = vectorstore.similarity_search_by_vector(embedding_vector, k=top_k)
    reference = {}
    for i, page in enumerate(docs):
        # match_content_list.append(page.page_content)
        reference[i] = {
            "reference_content": page.page_content,
            "metadata": page.metadata
        }


    template = """
    ### INSTRUCTION: 你是一位資深的監察院資料查詢專家。你的目標是解釋以下 REF 資料，用簡潔準確的方式説明其他人瞭解查詢到的相關內容。",
    ### REF: {reference}
    ### USER: {query}
    ### ASSISTANT: """
    prompt = PromptTemplate.from_template(template)
    chain = prompt | llm
    data = {
          "reference": " ".join(reference[0]['reference_content']),
          "query": query,
      }
    return chain, data, reference

query = "行政院原住民委員會的相關資訊"

chain, data, reference = query_similarity_content(query)
for s in chain.stream(data):
    print(s.content, end="", flush=True)
    
print("\n資料來源:", reference[0]['metadata']['document'])

c:\Users\sean.chang\AppData\Local\anaconda3\envs\project_envelope\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


行政院原住民委員會將原住民電視頻道的節目製作委託給公共電視文化事業基金會辦理。
資料來源: 52587.pdf


In [67]:
query = "行政院原住民委員會的相關資訊"

embedding_vector = embeddings.embed_query(query)
docs = vectorstore.similarity_search_with_score_by_vector(embedding_vector, k=2)
reference = {}
for i, page in enumerate(docs):
    # match_content_list.append(page.page_content)
    reference[i] = {
        "reference_content": page[0].page_content,
        "metadata": page[0].metadata,
        "score": page[1]
    }

In [68]:
reference

{0: {'reference_content': '行政院客家委員會、行政院原住民族委員會及僑務委員會將客家電視、原住民電視及臺灣宏觀電視等頻道節目之製播，依政府採購法委由財團法人公共電視文化事業基金會辦理，雖非無據，惟造成公視基金會與客委會、原民會及僑委會針對上開頻道節目之製播各有立場、互生歧見，致紛爭不斷，詎行政院新聞局坐視上開爭端，非但未積極研修法令，亦未尋求妥適解決辦法，核有怠失，爰依法提案糾正',
  'metadata': {'document': '52587.pdf',
   'target': '行政院新聞局',
   'fact': '行政院客家委員會、行政院原住民族委員會及僑務委員會(下稱客委會、原民會及僑委會)將客家電視、原住民電視及臺灣宏觀電視(下稱客視、原視及宏觀電視)等頻道節目之製播，依政府採購法委由財團法人公共電視文化事業基金會(下稱公視基金會)辦理，雖非無據，惟造成公視基金會與客委會、原民會及僑委會針對上開頻道節目之製播各有立場、互生歧見，致紛爭不斷，詎行政院新聞局(下稱新聞局)坐視上開爭端，非但未積極研修法令，亦未尋求妥適解決辦法，核有怠失，茲提出事實與理由於后一、據陳訴人指陳略以，95年無線公股處理條例於立法院三讀通過，目的在使黨、政、軍退出媒體，95年7月17日行政院召開「研商『無電視事業公股處理條例』規範原民台、客家台及宏觀頻道交由公共電視台製播2相關機關間溝通協調機制之建立問題」會議，會中決議「其製播所需經費之移撥(無政府採購法之適用問題)…」詎客委會、原民會及僑委會未依該決議辦理，仍依政府採購法，以標案形式由公視基金會辦理，實非合理又公視基金會辦理「原、客、宏觀頻道之計畫與製播」，並無獨立之經費得以辦理，而係由政府部門辦理審查、驗收，始得請領款項之情況，違反無線公股處理條例黨政軍退出媒體之精神，造成公共電視新聞自主性、專業性之箝制，詎該公股處理條例對「經費」疏漏等問題，迄未立法補正，亦造成公共電視台(下稱公視)財務經營之困境目前公視代為製播「原、客、宏觀頻道」，亦遭客委會、原民會及僑委會等延宕付款，更造成公視財務經營困難等語云云二、依據「無線電視事業公股處理條例」第14條第3項所定「政府編列預算招標採購或設置之客家電視、原住民電視、台灣宏觀電視等頻道節目之製播，應於本條例公布施行後之次年度起，『交由』公視基金會辦理」目

In [72]:
template = """
### INSTRUCTION: 你是一位資深的監察院資料查詢專家。你的目標是解釋以下 REF 資料，用簡潔準確的方式説明其他人瞭解查詢到的相關內容。",
### REF: {reference}
### USER: {query}
### ASSISTANT: """
prompt = PromptTemplate.from_template(template)

chain = prompt | llm

data = {
        "reference": "".join(reference[0]['reference_content']),
        "query": query,
    }

In [73]:
response = chain.invoke(data)

c:\Users\sean.chang\AppData\Local\anaconda3\envs\project_envelope\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


In [28]:
response.content

'行政院原住民委員會將原住民電視頻道節目的製播委託給公共電視文化事業基金會辦理。然而，原民會與公視基金會對於頻道節目的製播立場不同，導致紛爭不斷。行政院新聞局未能積極處理此爭端，因此監察院提出糾正案。'

In [75]:
response_data = {}
for i, page in enumerate(docs):
    # match_content_list.append(page.page_content)
    # reference[i] = {
    #     "reason": page.page_content,
    #     "metadata": page.metadata
    # }

    template = """
    ### INSTRUCTION: 你是一位資深的監察院案件資料專家。你的目標是對以下 REF 資料進行摘要。只要提供摘要結果就好。`
    ### REF: {reference}
    ### ASSISTANT: """
    prompt = PromptTemplate.from_template(template)
    chain = prompt | llm
    data = {
        "reference": page[0].page_content
    }


    llm_response = chain.invoke(data)

    response_data[i] = {
        "summary": llm_response.content,
        "source": page[0].metadata['document'],
        "score": page[1]
    }

c:\Users\sean.chang\AppData\Local\anaconda3\envs\project_envelope\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
c:\Users\sean.chang\AppData\Local\anaconda3\envs\project_envelope\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


In [76]:
response_data

{0: {'summary': '監察院糾正行政院新聞局怠忽職守，未適切處理公視基金會與客委會、原民會、僑委會間關於客家電視、原住民電視和臺灣宏觀電視節目製播的爭議。',
  'source': '52587.pdf',
  'score': 0.4295517},
 1: {'summary': '監察院糾正行政院退輔會因未蒐集商情資訊，片面指定採購專利設備，決策草率。此外，其所屬單位辦理污水處理工程時曲解採購法規，不當發包、擴增發包範圍，並由專利授權廠商決定發包對象，違反政府採購法相關規定。監察院認為退輔會未盡督導之責，有嚴重疏失。',
  'source': '37008.pdf',
  'score': 0.29004452}}

In [80]:
response_data[i] = {
    "summary": llm_response.content,
    "source": page[0].metadata['document'],
    "score": f"{page[1]:.2f}"
}
response_data

{0: {'summary': '監察院糾正行政院新聞局怠忽職守，未適切處理公視基金會與客委會、原民會、僑委會間關於客家電視、原住民電視和臺灣宏觀電視節目製播的爭議。',
  'source': '52587.pdf',
  'score': 0.4295517},
 1: {'summary': '監察院糾正行政院退輔會因未蒐集商情資訊，片面指定採購專利設備，決策草率。此外，其所屬單位辦理污水處理工程時曲解採購法規，不當發包、擴增發包範圍，並由專利授權廠商決定發包對象，違反政府採購法相關規定。監察院認為退輔會未盡督導之責，有嚴重疏失。',
  'source': '37008.pdf',
  'score': '0.29'}}